# Imports & setup

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
features = ['Was_home',
 'Rating_difference',
 'Min_2',
 'Gls_2',
 'Ast_2',
 'xG_2',
 'xA_2',
 'Team_CS_2',
 'Team_score_2',
 'Opp_score_2',
 'Team_result_2',
 'FPL_points_2',
 'Min_4',
 'Gls_4',
 'Ast_4',
 'xG_4',
 'xA_4',
 'Team_CS_4',
 'Team_score_4',
 'Opp_score_4',
 'Team_result_4',
 'FPL_points_4',
 'Min_30',
 'Gls_30',
 'Ast_30',
 'xG_30',
 'xA_30',
 'Team_CS_30',
 'Team_score_30',
 'Opp_score_30',
 'Team_result_30',
 'FPL_points_30',
 'DEF',
 'FWD',
 'MID']


info = ["Name_original", "GW", "Season", "Team", "Opponent", "Was_home", "Team_rating", "Opp_rating", "FPL_pos", "Price"]
to_predict = ["FPL_points"]

In [4]:
len(features)

35

# Reading from files

In [5]:
df = pd.read_csv("data/upcoming/upcoming_fixtures.csv")
CURRENT_SEASON = "2023-24"

In [7]:
X_test = df[df["Season"] == CURRENT_SEASON][features]
# .reset_index(drop=True)

In [8]:
df.columns

Index(['Unnamed: 0', 'Ast_2', 'Ast_30', 'Ast_4', 'DEF', 'FPL_points',
       'FPL_points_2', 'FPL_points_30', 'FPL_points_4', 'FPL_pos', 'FWD', 'GW',
       'Gls_2', 'Gls_30', 'Gls_4', 'MID', 'Min_2', 'Min_30', 'Min_4',
       'Name_original', 'Opp_rating', 'Opp_score_2', 'Opp_score_30',
       'Opp_score_4', 'Opponent', 'Rating_difference', 'Season', 'Team',
       'Team_CS_2', 'Team_CS_30', 'Team_CS_4', 'Team_rating', 'Team_result_2',
       'Team_result_30', 'Team_result_4', 'Team_score_2', 'Team_score_30',
       'Team_score_4', 'Was_home', 'xA_2', 'xA_30', 'xA_4', 'xG_2', 'xG_30',
       'xG_4', 'xP', 'Finished', 'Date', 'Price'],
      dtype='object')

In [9]:
X_test.columns

Index(['Was_home', 'Rating_difference', 'Min_2', 'Gls_2', 'Ast_2', 'xG_2',
       'xA_2', 'Team_CS_2', 'Team_score_2', 'Opp_score_2', 'Team_result_2',
       'FPL_points_2', 'Min_4', 'Gls_4', 'Ast_4', 'xG_4', 'xA_4', 'Team_CS_4',
       'Team_score_4', 'Opp_score_4', 'Team_result_4', 'FPL_points_4',
       'Min_30', 'Gls_30', 'Ast_30', 'xG_30', 'xA_30', 'Team_CS_30',
       'Team_score_30', 'Opp_score_30', 'Team_result_30', 'FPL_points_30',
       'DEF', 'FWD', 'MID'],
      dtype='object')

In [10]:
file = open("models/GBR.pkl",'rb')
model = pickle.load(file)

In [11]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=5,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

# Predicting

In [12]:
def adjust_for_injuries(df):
    active_players = pd.read_csv("data/misc/active_players.csv")
    injured_players = active_players[active_players["chance_of_playing_this_round"] == 0]["name"].to_list()
    df.loc[df["Name_original"].isin(injured_players),"Pred"] = 0
    return df

In [13]:
def get_predictions(model, df, X, all_remaining=False):
    # make predictions on the test data and glues them to the rest of the dataframe
    predictions = model.predict(X)
    df_predictions = df[(df["Season"] == CURRENT_SEASON)].reset_index(drop=True)
        
    print(len(predictions), df_predictions.shape)
        
    df_predictions.loc[:, "Pred"] = predictions
    preds = df_predictions[info + ["Pred"]]
    
    preds = adjust_for_injuries(preds).sort_values(by=["Pred"], ascending = False)
    
    return preds

In [14]:
preds = get_predictions(model, df, X_test)

13517 (13517, 49)


C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [15]:
# preds[(preds["FPL_pos"] == "GK") & (preds["GW"] == 2)].head(30)

In [20]:
# next gameweek
preds[preds["GW"] == 3].head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
9467,Mohamed Salah,3.0,2023-24,Liverpool,Newcastle Utd,0.0,1943.976074,1887.064819,MID,125.0,6.210399
8673,Marcus Rashford,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,90.0,5.374663
1254,Ben Mee,3.0,2023-24,Brentford,Crystal Palace,1.0,1822.547119,1763.994995,DEF,50.0,4.888554
10946,Reece James,3.0,2023-24,Chelsea,Luton,1.0,1791.018188,1602.327637,DEF,55.0,4.887369
4284,Folarin Balogun,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,FWD,45.0,4.742891
7802,Leandro Trossard,3.0,2023-24,Arsenal,Fulham,1.0,1921.462769,1745.365723,MID,70.0,4.575523
4019,Ethan Pinnock,3.0,2023-24,Brentford,Crystal Palace,1.0,1822.547119,1763.994995,DEF,45.0,4.531808
11098,Rico Henry,3.0,2023-24,Brentford,Crystal Palace,1.0,1822.547119,1763.994995,DEF,45.0,4.372695
1632,Bruno Borges Fernandes,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,MID,85.0,4.330943
8068,Lisandro Martínez,3.0,2023-24,Manchester Utd,Nott'ham Forest,1.0,1877.037842,1671.868774,DEF,50.0,4.317441


In [17]:
# all preds
preds.head(20)

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Pred
9500,Mohamed Salah,36.0,2023-24,Liverpool,Tottenham,1.0,1943.976074,1817.656738,MID,125.0,7.163340
9474,Mohamed Salah,10.0,2023-24,Liverpool,Nott'ham Forest,1.0,1943.976074,1671.868774,MID,125.0,7.163340
9495,Mohamed Salah,31.0,2023-24,Liverpool,Sheffield United,1.0,1943.976074,1637.473755,MID,125.0,7.163340
9494,Mohamed Salah,30.0,2023-24,Liverpool,Brighton,1.0,1943.976074,1832.364258,MID,125.0,7.163340
9466,Mohamed Salah,2.0,2023-24,Liverpool,Bournemouth,1.0,1943.976074,1662.536621,MID,125.0,7.163340
9468,Mohamed Salah,4.0,2023-24,Liverpool,Aston Villa,1.0,1943.976074,1816.929932,MID,125.0,7.163340
9470,Mohamed Salah,6.0,2023-24,Liverpool,West Ham,1.0,1943.976074,1777.551270,MID,125.0,7.163340
9473,Mohamed Salah,9.0,2023-24,Liverpool,Everton,1.0,1943.976074,1699.505859,MID,125.0,7.163340
9490,Mohamed Salah,26.0,2023-24,Liverpool,Luton,1.0,1943.976074,1602.327637,MID,125.0,7.163340
9502,Mohamed Salah,38.0,2023-24,Liverpool,Wolves,1.0,1943.976074,1715.631348,MID,125.0,7.163340


In [18]:
# points all season
pd.DataFrame(preds.groupby(["Name_original"])["Pred"].sum().sort_values(ascending=False)).head(30)

,Pred
Name_original,
Mohamed Salah,248.576279
Callum Wilson,173.694275
Kieran Trippier,170.466080
Marcus Rashford,166.249252
Erling Haaland,158.209091
Leandro Trossard,154.088455
Ollie Watkins,152.676620
Bruno Borges Fernandes,145.878296
Miguel Almirón Rejala,143.786194


# Saving to file

In [19]:
preds.to_csv("predictions/preds_next_season.csv", index=False)